# Genbank phylogeny from multiple genes

In [1]:
# importing everything we'll need
from Bio import SeqIO
from Bio.Blast.Applications import NcbiblastnCommandline
import os, glob, itertools, tarfile, subprocess
import pandas as pd
import numpy as np

Sequences obtained using code in the 'download_accessions.ipynb'

In [2]:
def create_dir(dir_name):
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)

In [79]:
def setup_taxdb():
    taxdb_dir = "{}/taxdb/".format(os.getcwd())
    if not (os.path.exists("{}/taxdb.btd".format(taxdb_dir)) and os.path.exists("{}/taxdb.bti".format(taxdb_dir))):
        print("taxdb files (taxdb/taxdb.btd e taxdb/taxdb.bti) not found. Extracting taxdb database...")
        create_dir(taxdb_dir)
        tarfile.open("{}/taxdb.tar.gz".format(os.getcwd()), "r:*").extractall(taxdb_dir)
    os.putenv("BLASTDB", taxdb_dir)
setup_taxdb()

In [80]:
def run_blastn(fasta):
    blast_output = "{}.blast".format(os.path.splitext(fasta)[0]) #Get fasta name without extension
    #blastn = NcbiblastnCommandline(cmd="blastn", query=fasta, db='nr', remote=True, outfmt="6 qseqid staxids sscinames sacc sseq",
    #                     max_target_seqs=5000, entrez_query="Formicidae [Organism]", out=blast_output)
    subprocess.run('blastn -query {} -db nr -max_target_seqs 5000 -remote \
    -entrez_query "Formicidae [Organism]" -outfmt "6 qseqid staxids sscinames sacc sseq"\
    -out {}'.format(fasta, blast_output), shell=True)

In [84]:
def run_all_blast_searches(directory):
    setup_taxdb()
    extensions = [".fa", ".fasta"]
    for extension in extensions:
        for fasta in glob.glob("{}*{}".format(directory, extension)):
            print(fasta)
            run_blastn(fasta)

In [3]:
# Finallly running blast for all seeds
# Não roda com sequências que tenham Ns

for directory in glob.glob("{}/seed_fasta/*/".format(os.getcwd())):
    print(directory)
#    run_all_blast_searches(directory)

/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/EF1/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/EF2/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/18S/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/COX1/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/results/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/28S/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/abdA/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/LWR/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/WNT/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/CYTB/
/home/gabriel/Dropbox/repos/genbank_phylogeny/seed_fasta/12S/


Saving the blast results into dataframes:

In [4]:
def create_dataframe(blast_result):
    return pd.read_csv(blast_result, sep='\t',
               names=['qseqid', 'staxids', 'sscinames', 'sacc', 'sseq']) 
    
def extract_columns_dataframe(df):
    df = df[['qseqid', 'staxids', 'sscinames', 'sacc', 'sseq']]
    df['sseq'] = df['sseq'].apply(lambda x: x.replace('-', ''))
    #df['sseqlen'] = len(df['sseq'])
    df['sseqlen'] = df.apply(lambda row: len(row.sseq), axis = 1) 
    df = df[['qseqid', 'staxids', 'sscinames', 'sacc', 'sseqlen', 'sseq']]
    return df
    
#df = extract_columns_dataframe(create_dataframe('seed_fasta/COX1/Formica_fusca_coi.blast'))
#print(dir(df["sscinames"].str))
#print(df.sscinames.str.contains("\."))

#df.head(n=100)
#df.to_excel("blast.xlsx")

Now that we have the blast results in a dataframe, we can clean it in order to:

-  Remove rows with more than one taxid;
-  Sort dataframe (descending) for both taxid and sseqlen;
-  Keep only one record by taxid (the one with the longest sseqlen);
-  Remove species with unwanted characters (".", ":", and "-") 

In [5]:
def clean_dataframe(df):
    #clean_df = df.drop(df[df.staxids.str.contains(";")].index) # Removing rows with hybrid sequences (more than one taxid value)
    clean_df = df.dropna() # Removes rows with NaN value(s)
    #clean_df["staxids"] = pd.to_numeric(clean_df["staxids"])
    clean_df = clean_df.sort_values(by=["staxids", "sseqlen"], ascending=False) # Sorting dataframe by taxid and sseqlen (descending) - Guarantees that highest sseqlen will always be the first row for that taxid
    clean_df = clean_df.drop_duplicates(subset="staxids", keep='first') # Keeps only one record per txid. The one that has the highest sseqlen
    for non_valid_ssciname in ["\.", "\:", "\-", "\;"]:
        clean_df = clean_df[~clean_df.sscinames.str.contains(non_valid_ssciname)] # Removes columns with sscinames containing the characters in the list
    return clean_df

#clean_dataframe(df).to_csv("clean.csv")

Now that we have the functions to extract and clean the data, we have to concatenate the blast results into a single, final dataframe:

In [6]:
def merge_blast_dataframes(directory):
    blast_data = []
    for blast_result in glob.glob("{}/*.blast".format(directory)):
        blast_data.append(clean_dataframe(extract_columns_dataframe(create_dataframe(blast_result))))
    return pd.concat(blast_data)

Let's create a matrix with the percentage of hits shared between all sequences:

In [7]:
#Calculating the number of subject accessions shared between two species

def matrix_absolute_matches(blast_full_dataframe):
    blast_dict = {k: f.tolist() for k, f in blast_full_dataframe.groupby('qseqid')['sacc']}
    matrix_dict = {k: [] for k in blast_dict.keys()} #Empty 
    for pair in itertools.product(blast_dict.keys(), repeat=2): #All possible combinations
        match = 0
        for acc in blast_dict[pair[0]]:
            if acc in blast_dict[pair[1]]:
                match += 1
        matrix_dict[pair[0]].append(match)
    return pd.DataFrame(matrix_dict, index=matrix_dict.keys())

In [11]:
#Calculating the number of subject accessions shared between two species
from random import randint

def unique_and_total_matches(blast_full_dataframe): 
    full_dataframe = blast_full_dataframe[['qseqid', 'sacc']].groupby('qseqid').count().rename(columns={'sacc': 'total_subjects'})
    unique_dataframe = blast_full_dataframe[['qseqid', 'sacc']].drop_duplicates('sacc', keep=False).groupby('qseqid').count().rename(columns={'sacc': 'unique_subjects'})
    return pd.concat([full_dataframe, unique_dataframe], axis=1)
        
results_dir = "{}/seed_fasta/results".format(os.getcwd())
create_dir(results_dir + '/')
writer = pd.ExcelWriter("unique_and_total_matches.xlsx", engine = 'xlsxwriter')
with pd.ExcelWriter("unique_and_total_matches.xlsx", engine = 'xlsxwriter') as writer:
    for directory in glob.glob("{}/seed_fasta/*".format(os.getcwd())):
        gene_name = os.path.basename(directory)
        if gene_name != os.path.basename(results_dir):
            blast_full_dataframe = merge_blast_dataframes(directory)
            #writer = pd.ExcelWriter("unique_and_total_matches.xlsx", engine = 'xlsxwriter')
            unique_and_total_matches(blast_full_dataframe).to_excel(writer, sheet_name = gene_name)
    #writer
#writer.save()
#writer.close()

In [102]:
#Calculating the number of subject accessions shared between two species

def unique_and_total_matches(blast_full_dataframe, gene_name):
    all_accessions = blast_full_dataframe['sacc'].tolist()
    print("All: {}".format(len(all_accessions)))
    unique_accessions = blast_full_dataframe['sacc'].unique().tolist()
    print("Unique: {}".format(len(unique_accessions)))
    duplicates = len(blast_full_dataframe['sacc'])-len(blast_full_dataframe['sacc'].drop_duplicates())
    print("Duplicate: {}".format(duplicates)) 
    blast_dict = {k: f.tolist() for k, f in blast_full_dataframe.groupby('qseqid')['sacc']}
    for key in blast_dict.keys():
        total_matches = 0
        unique_matches = 0
        total_matches = len(blast_dict.get(key))
        for unique_acc in unique_accessions:
            if unique_acc in blast_dict.get(key):
                unique_matches += 1
                print(unique_matches)
                #print("{} in {}".format(unique_acc, key))
            else:
                #print("{} not in {}".format(unique_acc, key))
                pass
        print(key, total_matches, unique_matches)
   # data_dict = {k: [] for k in ["Total matches", "Unique matches"]} 
    
results_dir = "{}/seed_fasta/results".format(os.getcwd())
create_dir(results_dir + '/')
for directory in glob.glob("{}/seed_fasta/*".format(os.getcwd())):
    gene_name = os.path.basename(directory)
    if gene_name != os.path.basename(results_dir):
        #print("Getting subject seqs for {} seed".format(gene_name))
        full_dataframe = merge_blast_dataframes(directory)
        #print(full_dataframe)
        unique_and_total_matches(full_dataframe, gene_name)
        #global last_df
        #last_df = unique_and_total_matches(full_dataframe, gene_name)
        #last_df
    break
        #unique_and_total_matches(full_dataframe, gene_name).to_excel("{}/matches_report.xlsx".format(results_dir), index=False, sheet_name=gene_name)

All: 1974
Unique: 1644
Duplicate: 330
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
26

In [102]:
#Calculating the number of subject accessions shared between two species

def unique_and_total_matches(blast_full_dataframe, gene_name):
    all_accessions = blast_full_dataframe['sacc'].tolist()
    print("All: {}".format(len(all_accessions)))
    unique_accessions = blast_full_dataframe['sacc'].unique().tolist()
    print("Unique: {}".format(len(unique_accessions)))
    duplicates = len(blast_full_dataframe['sacc'])-len(blast_full_dataframe['sacc'].drop_duplicates())
    print("Duplicate: {}".format(duplicates)) 
    blast_dict = {k: f.tolist() for k, f in blast_full_dataframe.groupby('qseqid')['sacc']}
    for key in blast_dict.keys():
        total_matches = 0
        unique_matches = 0
        total_matches = len(blast_dict.get(key))
        for unique_acc in unique_accessions:
            if unique_acc in blast_dict.get(key):
                unique_matches += 1
                print(unique_matches)
                #print("{} in {}".format(unique_acc, key))
            else:
                #print("{} not in {}".format(unique_acc, key))
                pass
        print(key, total_matches, unique_matches)
   # data_dict = {k: [] for k in ["Total matches", "Unique matches"]} 
    
results_dir = "{}/seed_fasta/results".format(os.getcwd())
create_dir(results_dir + '/')
for directory in glob.glob("{}/seed_fasta/*".format(os.getcwd())):
    gene_name = os.path.basename(directory)
    if gene_name != os.path.basename(results_dir):
        #print("Getting subject seqs for {} seed".format(gene_name))
        full_dataframe = merge_blast_dataframes(directory)
        #print(full_dataframe)
        unique_and_total_matches(full_dataframe, gene_name)
        #global last_df
        #last_df = unique_and_total_matches(full_dataframe, gene_name)
        #last_df
    break
        #unique_and_total_matches(full_dataframe, gene_name).to_excel("{}/matches_report.xlsx".format(results_dir), index=False, sheet_name=gene_name)

All: 1974
Unique: 1644
Duplicate: 330
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
26

Lastly, let's create 3 final fasta files:

1. With all query sequences
2. With all unique subject sequences from the blast searches
1. With all query + subject sequences

In [10]:
def queries_to_multifasta(directory):
    #create_dir("final_seqs/")
    with open("./final_seqs/queries.fa", "w") as queries:
        for fasta in glob.glob("{}/*fa".format(directory)):
            for record in SeqIO.parse(fasta, "fasta"):
                queries.write(record.format("fasta"))

def subjects_to_multifasta(blast_full_dataframe, results_dir, gene_name):
    #create_dir("final_seqs/")
    unique_blast_subjects = blast_full_dataframe.sort_values(by=["sseqlen"], ascending=False).drop_duplicates(subset="staxids", keep='first') # Keep only largest sequence per taxid
    with open("{}/{}_subjects.fa".format(results_dir,gene_name), "w") as subjects:
        for row in unique_blast_subjects.itertuples(): # Awfully slow. Need to optimize this later
            subjects.write(">{}_{}\n{}\n".format(row.sscinames.replace(" ", "_"), row.sacc, row.sseq))
        #unique_blast_subjects.apply(lambda x: subjects.write(">{}_{}\n{}".format(x['sscinames'].replace(" ", "_"), x["sacc"], x['sseq'])))

After defining all these functions, it is time to run the pipeline:

In [121]:
# Finallly running blast for all seeds
# Não roda com sequências que tenham Ns

results_dir = "{}/seed_fasta/results".format(os.getcwd())
create_dir(results_dir + '/')
for directory in glob.glob("{}/seed_fasta/*".format(os.getcwd())):
    gene_name = os.path.basename(directory)
    if gene_name != os.path.basename(results_dir):
        print("Getting subject seqs for {} seed".format(gene_name))
        full_dataframe = merge_blast_dataframes(directory)
        full_dataframe.to_csv("{}_teste.csv".format(gene_name))
        subjects_to_multifasta(full_dataframe, results_dir, gene_name)
        matrix_absolute_matches(full_dataframe).to_excel("{}/{}_matrix.xlsx".format(results_dir,gene_name), 
                                                         index=False)

EF1
EF2
18S
COX1
28S
abdA
LWR
WNT
CYTB
12S


That's pretty much it!!! :)